# Optimize Acquisition Functions using torch.optim

In this tutorial, we show how to use PyTorch's `optim` module for optimizing botorch MC acquisition functions. This is useful if the acquisition function is stochastic in nature (caused by re-sampling the base samples when using the reparameterization trick, or if the model output itself is stochastic).

As discussed in the [CMA-ES tutorial](./optimize_with_cmaes.ipynb), for deterministic acquisition functions botorch by default uses quasi-second order methods (such as L-BFGS-B or SLSQP), which provide superior convergence speed in this situation. 

### Starting point

Let's assume for the purpose of this tutorial that
- `acq_function` is an instance of a botorch `MCAcquistionFunction`, for instance `qExpectedImprovement`
- the goal is to optimize a `q`-batch of size `q` in `d`-dim feature space in a hyperrectangle
- `bounds` is a `2 x d`-dim tensor, where `bounds[0, :]` (`bounds[1, :]`) are the lower (upper) bounds of the feasible hyperrectangle


We will perform optimization over `N` random intial `q`-batches in parallel. This is on order to account for the fact that the acquisition function is non-convex and as a result we may get stuck in local minima.

In [ ]:
import torch

In [ ]:
# generate N random q-batch initial conditions in the d-dim unit square
X = torch.rand(N, q, d)

# scale this to the constraint rectangle
X = bounds[0] + (bounds[1] - bounds[0]) * X

# we'll want gradients
X.requires_grad_(True);

If you have used PyTorch, the basic optimization loop should be quite familiar. Note though that there is a **key difference** here compared to training ML models: When training ML models, one typically computes the gradient of an empirical loss function w.r.t. the module's parameters, while here we take the gradient of the acquisition function w.r.t. to the candidate set.

Thus, when setting the optimizer from `torch.optim`, we **do not** add the acquisition function's parameters as parameters to optimize (that would be quite bad!).

In this example, we use a vanilla `Adam` optimizer with fixed learning rate for a fixed number of iterations in order to keep things simple. But you can get as fancy as you want with learning rate scheduling, early termination, etc.

One thing to note is that evaluating the acquisition function on the `N x q x d`-dim inputs means evaluating `N` `q`-batches in `t`-batch mode. The result of this is an `N`-dim tensor of acquisition function values, evaluated independently. To compute the gradient of the full input `X` via back-propagation, we can for convenience just compute the gradient of the sum of the losses. 

In [ ]:
# set up the optimizer, make sure to only pass in the candidate set here
optimizer = torch.optim.Adam([X], lr=0.05)

# run a basic optimization loop
for i in range(75):
    optimizer.zero_grad()
    # this performs batch evaluation, so this is an N-dim tensor
    losses = - acq_func(X)  # torch.optim tries to minimize
    loss = losses.sum()
    loss.backward()
    optimizer.step()
    # use your favorite convergence criterion here...